# パイプラインにおけるデータ品質チェックによるデータの隔離



本ノートブックでは、 DQX によるデータ品質チェックを実施して、データを隔離する方法を紹介します。


## Databricks 提供の DQX(Data Quality Framework)のインストール

In [0]:
%pip install databricks-labs-dqx -q
dbutils.library.restartPython()

## サンプルデータの準備

In [0]:
data = [
    {"id": 1, "amount": 100,  "order_date": "2025-01-01"},   # 正常
    {"id": 2, "amount": "二百","order_date": "2025-02-01"},  # amount と が数値に変換不可
    {"id": None, "amount": 300, "order_date": "2025-04-01"},  # order_no が null
    {"id": None, "amount": "四百", "order_date": "2025:08:24"},  # 全部 NG
]

In [0]:
# 文字型で読まない場合にデータが損失する事象を確認
bad_df = spark.createDataFrame(data)
bad_df.createOrReplaceTempView("_tmp")
bad_df = spark.sql(
    """
    SELECT
        TRY_CAST(id AS INT) AS id,
        TRY_CAST(amount AS DECIMAL(18,2)) AS amount,
        TRY_CAST(order_date AS date) AS order_date
        FROM
            _tmp
    """
)
display(bad_df)

In [0]:
# 文字型に読み込むことでデータを確認
string_schema = """
id STRING,
amount STRING,
order_date STRING
"""
df = spark.createDataFrame(data, string_schema)
display(df)

## データ品質チェックによるデータの隔離

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.labs.dqx.engine import DQEngine

In [0]:
# 品質チェック条件を記述。YAML でも記述可。
checks = [
    # 1) NOT NULL
    {
        "criticality": "error",
        "check": {"function": "is_not_null", "arguments": {"column": "id"}},
    },
    # 2) 一意性（id の重複を検出）
    {
        "criticality": "error",
        "check": {"function": "is_unique", "arguments": {"columns": ["id"]}},
    },
    # 3) データ型変換
    {
        "criticality": "error",
        "name": "amount_castable_to_decimal",
        "check": {
            "function": "sql_expression",
            "arguments": {
                "expression": "try_cast(amount as decimal(18,2)) IS NOT NULL OR amount IS NULL",
                "msg": "amount cannot be cast to DECIMAL(18,2)",
            },
        },
    },
    {
        "criticality": "error",
        "name": "order_date_castable_to_date",
        "check": {
            "function": "sql_expression",
            "arguments": {
                "expression": "try_cast(order_date as date) IS NOT NULL OR order_date IS NULL",
                "msg": "order_date cannot be cast to DATE",
            },
        },
    },
]

In [0]:
# データ品質によるデータ隔離を実施
dq = DQEngine(WorkspaceClient())
raw_of_output_df, raw_of_error_df = dq.apply_checks_by_metadata_and_split(df, checks)

print("-- GOOD (合格)")
display(raw_of_output_df)

print("--- BAD / QUARANTINE (不良・隔離)")
display(raw_of_error_df)

## Next Step

[Demos | DQX](https://databrickslabs.github.io/dqx/docs/demos/)にて、 DQX に対する理解を深めましょう。

![dqx](./.images/demos_dqx.png)


In [0]:
# end